# MERGE DATA

## modules:

Standard modules: 

In [10]:
import numpy as np
import pandas as pd

Visualization modules: 

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline
import vincent as vin
import seaborn as sb

Machine-learning modules: 

In [12]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.cross_validation import cross_val_score, train_test_split

from patsy import dmatrices 

## 1. kaggle data

In [29]:
# taste_profile_song_to_tracks
song_to_track = pd.read_table('/Users/juliaengel-dagher/Downloads/taste_profile_song_to_tracks.txt', sep='\t', names =['song_ids','track_id'])

In [30]:
song_to_track.head()
# I will need the track ids for merging genrea and lyrics data

,song_ids,track_id
0,SOAAADD12AB018A9DD,TRNCENP12903C9EF3A
1,SOAAADE12A6D4F80CC,TRSKKFK128F148B615
2,SOAAADF12A8C13DF62,TRCQMSP128F428A6F7
3,SOAAADZ12A8C1334FB,TRMDNZY128F425A532
4,SOAAAFI12A6D4F9C66,TRZEXLQ128F1491D17


In [31]:
len(song_to_track)

386213

In [15]:
#kaggle_visible_evaluation_triplets
#visible part of the testing data
user_behavior = pd.read_table('/Users/juliaengel-dagher/Downloads/kaggle_visible_evaluation_triplets.txt', sep='\t', names =['user_id','song_ids','play_count'])

In [16]:
user_behavior.head()

,user_id,song_ids,play_count
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1


Next, I am creating a pivot table so that we have one line per song_ids summarixing the number of users who listened to the song and how many times they listened to it. 

In [22]:
funcs = {'play_count':['sum'], 'user_id':['count']}

In [23]:
behavior_df = user_behavior.groupby('song_ids').agg(funcs)

In [24]:
behavior_df.head()

,play_count,user_id
,sum,count
song_ids,,
SOAAAFI12A6D4F9C66,2,2
SOAAAGK12AB0189572,2,1
SOAAAGQ12A8C1420C8,90,33
SOAAAMT12AB018C9C4,1,1
SOAAAQN12AB01856D3,2,2


In [26]:
len(behavior_df)

163206

In [33]:
behavior_df_wtrackids= pd.read_csv('/Users/juliaengel-dagher/Downloads/behavior_df.csv', sep=',')

In [34]:
behavior_df_wtrackids.head()

,song_ids,play_count_sum,user_id_count
0,SOAAAFI12A6D4F9C66,2,2
1,SOAAAGK12AB0189572,2,1
2,SOAAAGQ12A8C1420C8,90,33
3,SOAAAMT12AB018C9C4,1,1
4,SOAAAQN12AB01856D3,2,2


In [57]:
behavior_df_wtrackids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163206 entries, 0 to 163205
Data columns (total 4 columns):
song_ids          163206 non-null object
track_id          162183 non-null object
play_count_sum    163206 non-null int64
user_id_count     163206 non-null int64
dtypes: int64(2), object(2)
memory usage: 6.2+ MB


In [58]:
behavior_df_wtrackids.describe()

,play_count_sum,user_id_count
count,163206.000000,163206.000000
mean,28.334375,8.890194
std,215.826789,46.780258
min,1.000000,1.000000
25%,1.000000,1.000000
50%,5.000000,2.000000
75%,15.000000,6.000000
max,35432.000000,5043.000000


In [35]:
behavior_df_wtrackids = pd.merge(behavior_df_wtrackids, song_to_track, on='song_ids',how='inner')

In [36]:
behavior_df_wtrackids.head()

,song_ids,play_count_sum,user_id_count,track_id
0,SOAAAFI12A6D4F9C66,2,2,TRZEXLQ128F1491D17
1,SOAAAGK12AB0189572,2,1,TRDZUFJ12903CE29FC
2,SOAAAGQ12A8C1420C8,90,33,TRJWZII128F92CA924
3,SOAAAMT12AB018C9C4,1,1,TRYIOYF12903CD4E73
4,SOAAAQN12AB01856D3,2,2,TRATLOJ128F9308ABB


In [38]:
behavior_df_wtrackids = behavior_df_wtrackids[['song_ids', 'track_id', 'play_count_sum', 'user_id_count']]

In [39]:
behavior_df_wtrackids.head()

,song_ids,track_id,play_count_sum,user_id_count
0,SOAAAFI12A6D4F9C66,TRZEXLQ128F1491D17,2,2
1,SOAAAGK12AB0189572,TRDZUFJ12903CE29FC,2,1
2,SOAAAGQ12A8C1420C8,TRJWZII128F92CA924,90,33
3,SOAAAMT12AB018C9C4,TRYIOYF12903CD4E73,1,1
4,SOAAAQN12AB01856D3,TRATLOJ128F9308ABB,2,2


In [41]:
behavior_df_wtrackids.to_csv('behavior_df_final.csv', sep=',')

## 2. genre data

http://www.ifs.tuwien.ac.at/mir/msd/

In [45]:
genres = pd.read_table('http://www.ifs.tuwien.ac.at/mir/msd/partitions/msd-MAGD-genreAssignment.cls',sep='\t')

In [46]:
genres.head()

,TRAAAAK128F9318786,Pop_Rock
0,TRAAAAV128F421A322,Pop_Rock
1,TRAAAAW128F429D538,Rap
2,TRAAABD128F429CF47,Pop_Rock
3,TRAAACV128F423E09E,Pop_Rock
4,TRAAADT12903CCC339,Easy_Listening


In [47]:
genres.columns=['track_id','genre_id']

In [48]:
genres.head()

,track_id,genre_id
0,TRAAAAV128F421A322,Pop_Rock
1,TRAAAAW128F429D538,Rap
2,TRAAABD128F429CF47,Pop_Rock
3,TRAAACV128F423E09E,Pop_Rock
4,TRAAADT12903CCC339,Easy_Listening


In [49]:
behavior_df_wgenre = pd.merge(behavior_df_wtrackids, genres, on='track_id',how='inner')

In [50]:
behavior_df_wgenre.head()

,song_ids,track_id,play_count_sum,user_id_count,genre_id
0,SOAAAMT12AB018C9C4,TRYIOYF12903CD4E73,1,1,Pop_Rock
1,SOAAAXN12A8C13A70C,TRXWDIG128F4274BF1,5,1,Latin
2,SOAABLP12A6D4F8861,TRXOSZE128F148A158,1,1,Pop_Rock
3,SOAABRT12A6D4F7940,TRUGBDI128F14548CD,13,2,International
4,SOAABVA12AC3DF673F,TRXVIGQ12903CE7F6E,4,3,Pop_Rock


In [ ]:
f=df.plot(df[['genre_id']].grouby(),df[['play_count_sum']].sum())

In [51]:
len(behavior_df_wgenre)

84102

## 4. hdf5 song attributes

In [52]:
hdf5_df = pd.read_csv('music_project.csv')

In [53]:
behavior_df_whdf5 = pd.merge(behavior_df_wgenre, hdf5_df, on='song_ids',how='inner')

In [54]:
behavior_df_whdf5.head()

,song_ids,track_id,play_count_sum,user_id_count,genre_id,Unnamed: 0,artist_familiarty,artist_hotness,artist_names,song_bars,...,song_durations,song_energies,song_hotness,song_keys,song_modes,song_realease_years,song_tatum,song_tempo,song_time_signatures,song_titles
0,SOAAEHR12A6D4FB060,TRAYPFH128E07937C3,16,11,Pop_Rock,7181,0.806359,0.545789,Soundgarden,[ 0.85125 2.69826 4.53846 6.3912 ...,...,415.81669,0,0.654068,0,1,1991,[ 1.56890000e-01 3.88730000e-01 6.2056000...,131.272,4,Slaves & Bulldozers
1,SOABLAF12AB018E1D9,TRABUMO12903CBE25F,66,15,Pop_Rock,546,0.796337,0.582922,Alice In Chains,[ 0.92741 3.15716 5.38085 7.60259 ...,...,256.52200,0,0.824433,10,0,2009,[ 8.94000000e-02 3.69200000e-01 6.4900000...,107.845,4,Lesson Learned
2,SOACEDS12A6701EAAA,TRASNOT128E0791785,5,4,Pop_Rock,5412,0.712745,0.470700,Semisonic,[ 0.67473 3.1441 5.58583 8.0254 ...,...,232.25424,0,0.612055,0,1,1998,[ 0.36436 0.67473 0.98823 1.29676 ...,97.995,4,Gone To The Movies
3,SOAECHX12A6D4FC3D9,TRACSBL128F42480E8,17,3,Pop_Rock,822,0.724864,0.524449,The Presets,[ 1.48499 3.48481 5.48587 7.48445 ...,...,271.85587,0,0.663125,4,1,2008,[ 2.40190000e-01 4.89400000e-01 7.3737000...,120.119,4,Yippiyo-Ay
4,SOAEGMV12A8C1455F2,TRAVZWX128F92EC110,1,1,Pop_Rock,6418,0.651773,0.524477,Berlin,[ 1.45526 3.73822 6.01853 8.33562 ...,...,255.32036,0,0.270776,11,1,2002,[ 0.58706 0.88027 1.16776 1.45526 ...,104.040,4,Blink Of An Eye (Alternative)


In [55]:
len(behavior_df_whdf5)

864

Unfortunately, this inner join, cuts out too much information and we are left with a very small dataset. 

## 5. AWS EC2 HDF5 files

Therefore, let's try the AWS approach. 
The million song set already exists on AWS as a public data set:
https://aws.amazon.com/datasets/million-song-dataset/

I am following these instructions to create a volume, attach it to an instance and then mount it. 
http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-public-data-sets.html

## 6. lyrics data

Reading in the lyrics information by song id and saving it as a csv in order to merge it with the larger data set by song id. http://labrosa.ee.columbia.edu/millionsong/musixmatch

- song lyrics in bag-of-words format for the million song data set
- provided as a SQLlite database
- python code provided https://github.com/tbertinmahieux/MSongsDB/blob/master/Tasks_Demos/Lyrics/mxm_dataset_to_db.py

In [ ]:
For this set I needed to use AWS. I created an EC2 instance in AWS. 
#terminal: 
# I copied the pyhton file and the databases provided by the million song set to put the lyrics data into a sqlite database
scp -i my_key.pem MSongsDB/Tasks_Demos/Lyrics/mxm_dataset_to_db.py ec2-user@ec2-52-87-214-185.compute-1.amazonaws.com:
scp -i my_key.pem track_metadata.db ec2-user@ec2-52-87-214-185.compute-1.amazonaws.com:~

In [ ]:
#terminal: 
#let's run my AWS instance
ssh -i "my_key.pem" ec2-user@ec2-52-87-214-185.compute-1.amazonaws.com

#then I ran the python file
python mxm_dataset_to_db.py

#the lyrics data is now in the sqlite database in AWS EC2

This is the code provided by http://labrosa.ee.columbia.edu/millionsong/blog/11-4-11-musixmatch-dataset-connecting-lyrics

In [ ]:
terminal: 
    python
    import sqlite3
    conn = sqlite3.connect('mxm_dataset.db')
    res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
    res.fetchall()
    res = conn.execute("SELECT word FROM words")
    len(res.fetchall())

In [ ]:
#remebere to stop the AWS instance 
#smile

## Save final dataframe as csv